In [7]:
import datetime

In [8]:
cliente = 'PR'
start_date = datetime.datetime(2022, 7, 1,0,0)
end_date = datetime.datetime(2024, 12, 31,23,0)

In [9]:
year = 2024
month = 10
operation = 'estim' # estim o verif

In [10]:
import pandas as pd

In [11]:
from config.config_loader import Config
config = Config()

In [12]:
path_alias_elec = config.get_resolved_path('data_path.elec.customers.alias_elec')
path_load_parquet = config.get_resolved_path("data_path.elec.processed.load_parquet")
spot_formatted_path = config.get_resolved_path("data_path.elec.processed.spot_formatted")
path_tope_gas = config.get_resolved_path('data_path.elec.processed.tope_gas')
captured_price_path = config.get_path("data_path.elec.customers.customer_single.captured_price", customer_id=cliente)
solar_captured_price_path = config.get_path("data_path.elec.customers.customer_single.solar_captured_price", customer_id=cliente)
grouped_load_hourly_path = config.get_path("data_path.elec.customers.customer_single.grouped_load_hourly", customer_id=cliente)
path_tope_gas

'c:\\Users\\mikel.perez\\0_Python_Projects - copia\\0_Python_projects_v2\\data\\processed\\elec\\tope_gas.xlsx'

In [13]:
df_alias = pd.read_excel(path_alias_elec)
df_alias_filtered = df_alias[df_alias['cliente'] == cliente]
cups_list = df_alias_filtered['cups'].values
cups_list

array(['ES0031408040511001BG', 'ES0031405931106001DF',
       'ES0031101483796004QM', 'ES0021000007935634YX',
       'ES0021000001562230MK', 'ES0021000005737638YD',
       'ES0021000004838105LY', 'ES0022000007945080PQ',
       'ES0021000004838111LN', 'ES0022000008103260DW',
       'ES0022000009138573BQ', 'ES0022000009138572BS',
       'ES0022000009138571BZ', 'ES0031104220087001BT',
       'ES0031103656433002ED', 'ES0031103656433003EX',
       'ES0031104081510001GR', 'ES0031104004641001AZ',
       'ES0021000021847556HZ', 'ES0021000012579474HN',
       'ES0021000012579469HF', 'ES0021000012579468HY',
       'ES0031103666028002EF', 'ES0031102982960005WV'], dtype=object)

In [14]:
spot_df = pd.read_pickle(spot_formatted_path)
spot_df = spot_df[(spot_df['datetime'] >= start_date) & (spot_df['datetime'] <= end_date)]
spot_df = spot_df.copy()
spot_df.loc[:, 'dup_count'] = spot_df.groupby('datetime').cumcount()
spot_df

,datetime,value,date,month,year,time,periodo,dup_count
74467,2022-07-01 00:00:00,147.64,2022-07-01,7,2022,00:00:00,6,0
74468,2022-07-01 01:00:00,146.48,2022-07-01,7,2022,01:00:00,6,0
74469,2022-07-01 02:00:00,143.70,2022-07-01,7,2022,02:00:00,6,0
74470,2022-07-01 03:00:00,142.40,2022-07-01,7,2022,03:00:00,6,0
74471,2022-07-01 04:00:00,144.96,2022-07-01,7,2022,04:00:00,6,0
...,...,...,...,...,...,...,...,...
96422,2024-12-31 19:00:00,159.44,2024-12-31,12,2024,19:00:00,1,0
96423,2024-12-31 20:00:00,156.10,2024-12-31,12,2024,20:00:00,1,0
96424,2024-12-31 21:00:00,150.85,2024-12-31,12,2024,21:00:00,1,0
96425,2024-12-31 22:00:00,144.40,2024-12-31,12,2024,22:00:00,2,0


In [15]:
df_tope_gas = pd.read_excel(path_tope_gas, skiprows=1)
df_tope_gas = df_tope_gas[['datetime', 'value']]
df_tope_gas.columns = ['datetime', 'tope_gas']
df_tope_gas['datetime'] = df_tope_gas['datetime'].apply(lambda x: x[:-7])
df_tope_gas['datetime'] = pd.to_datetime(df_tope_gas['datetime'])
df_tope_gas = df_tope_gas.copy()
df_tope_gas.loc[:, 'dup_count'] = df_tope_gas.groupby('datetime').cumcount()
df_tope_gas.tail(1)

,datetime,tope_gas,dup_count
22680,2024-12-31 23:00:00,0.0,0


In [54]:
df_merge = pd.merge(df_tope_gas, spot_df, on=['datetime', 'dup_count'])
df_merge['precio'] = df_merge['value'] + df_merge['tope_gas']
# df_merge.drop(['value', 'tope_gas'], axis=1, inplace=True)
# df_merge = df_merge[['datetime', 'dup_count', 'precio']]
# df_merge.columns = ['datetime', 'dup_count', 'value']
df_merge

,datetime,tope_gas,dup_count,value,date,month,year,time,periodo,precio
0,2022-07-01 00:00:00,97.86,0,147.64,2022-07-01,7,2022,00:00:00,6,245.50
1,2022-07-01 01:00:00,104.46,0,146.48,2022-07-01,7,2022,01:00:00,6,250.94
2,2022-07-01 02:00:00,111.08,0,143.70,2022-07-01,7,2022,02:00:00,6,254.78
3,2022-07-01 03:00:00,116.19,0,142.40,2022-07-01,7,2022,03:00:00,6,258.59
4,2022-07-01 04:00:00,120.07,0,144.96,2022-07-01,7,2022,04:00:00,6,265.03
...,...,...,...,...,...,...,...,...,...,...
21955,2024-12-31 19:00:00,0.00,0,159.44,2024-12-31,12,2024,19:00:00,1,159.44
21956,2024-12-31 20:00:00,0.00,0,156.10,2024-12-31,12,2024,20:00:00,1,156.10
21957,2024-12-31 21:00:00,0.00,0,150.85,2024-12-31,12,2024,21:00:00,1,150.85
21958,2024-12-31 22:00:00,0.00,0,144.40,2024-12-31,12,2024,22:00:00,2,144.40


In [55]:
df = pd.read_parquet(path_load_parquet)  
df = df[df['cups'].isin(cups_list)]
df = df[df['resolution'] == 'hourly']
df = df[(df['datetime'] >= start_date) & (df['datetime'] <= end_date)]

df['dup_count'] = df.groupby(['cups', 'datetime', 'resolution']).cumcount()

ds = df.groupby(['datetime', 'dup_count'])['load'].sum()
df_agg = pd.DataFrame(ds)
df_agg.reset_index(inplace=True)
df_agg.tail()

,datetime,dup_count,load
21957,2024-12-31 19:00:00,0,1524.0
21958,2024-12-31 20:00:00,0,1514.0
21959,2024-12-31 21:00:00,0,1525.0
21960,2024-12-31 22:00:00,0,1493.0
21961,2024-12-31 23:00:00,0,1479.0


In [56]:
df_final = pd.merge(df_agg, df_merge, how='left', on=['datetime', 'dup_count'])
df_final.head()

,datetime,dup_count,load,tope_gas,value,date,month,year,time,periodo,precio
0,2022-07-01 00:00:00,0,2968.0,97.86,147.64,2022-07-01,7.0,2022.0,00:00:00,6.0,245.50
1,2022-07-01 01:00:00,0,2794.0,104.46,146.48,2022-07-01,7.0,2022.0,01:00:00,6.0,250.94
2,2022-07-01 02:00:00,0,2661.0,111.08,143.70,2022-07-01,7.0,2022.0,02:00:00,6.0,254.78
3,2022-07-01 03:00:00,0,2635.0,116.19,142.40,2022-07-01,7.0,2022.0,03:00:00,6.0,258.59
4,2022-07-01 04:00:00,0,2577.0,120.07,144.96,2022-07-01,7.0,2022.0,04:00:00,6.0,265.03


In [57]:
# CAPTURED PRICE

ds1 = df_final.groupby([df_final['datetime'].dt.year, df_final['datetime'].dt.month])[['value', 'load']].apply(lambda row: (row['value'] * row['load']).sum() / row['load'].sum())
# ds1 = df_final.groupby([df_final['datetime'].dt.year, df_final['datetime'].dt.month])[['tope_gas', 'load']].apply(lambda row: (row['tope_gas'] * row['load']).sum() / row['load'].sum())
df1 = pd.DataFrame(ds1)
df1.index.names = ['year', 'month']
df1.reset_index(drop=False,inplace=True)
df1.columns = ['year', 'month', 'price']
df1.to_excel(captured_price_path)
# df1.to_excel(path_tope_gas)
df1

,year,month,price
0,2022,7,140.753836
1,2022,8,149.527228
2,2022,9,135.393630
3,2022,10,125.181936
4,2022,11,115.601525
5,2022,12,96.842629
6,2023,1,62.512071
7,2023,2,127.733191
8,2023,3,82.913248
9,2023,4,57.077839


In [58]:
# LOAD GROUPED BY HOUR

ds1 = df_final.groupby([df_final['datetime'].dt.year, df_final['datetime'].dt.month, df_merge['datetime'].dt.time])['load'].sum()
df1 = pd.DataFrame(ds1)
df1.index.names = ['year', 'month', 'time']
df1.reset_index(drop=False,inplace=True)
df1.columns = ['year', 'month', 'time', 'load']

df_pivoted = df1.pivot(index=['year', 'month'], columns='time', values='load').reset_index()

# Rename columns for better readability (optional)
df_pivoted.columns.name = None  # Remove the "time" name from columns
df_pivoted.rename(columns=lambda x: f'Hour_{x}' if isinstance(x, int) else x, inplace=True)
df_pivoted.to_excel(grouped_load_hourly_path)

df_pivoted.groupby(['year']).sum()

,month,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,...,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
year,,,,,,,,,,,,,,,,,,,,,
2022,57,485341.0,460170.0,439144.0,426103.0,424141.0,419711.0,426028.0,432671.0,463077.0,...,1130722.0,1142696.0,1132435.0,1104373.0,1068218.0,1003440.0,877168.0,762942.0,654433.0,536426.0
2023,78,402028.0,374476.0,346894.0,334710.0,331819.0,331102.0,345396.0,372354.0,412761.0,...,1010997.0,1018785.0,1008786.0,983507.0,970091.0,936681.0,825456.0,741198.0,585516.0,486762.0
2024,78,1119136.0,1002882.0,906696.0,876682.0,850684.0,840291.0,834833.0,836161.0,860469.0,...,2002814.0,2093857.0,2107886.0,2095452.0,2044449.0,1973569.0,1887811.0,1723055.0,1464480.0,1356712.0
